# Testing Fidelity With Trained Model
The purpose of this notebook is to experiment with the model trained in [this notebook](training_new_model.ipynb). Specifically, I will be performing the rough test described in **Entry 1** of [this journal page](../Journal/10_10_24.md).

In [2]:
### IMPORTS
import warnings
import os
from lambeq import BobcatParser, RemoveCupsRewriter, AtomicType, IQPAnsatz, TketModel
import numpy as np
from pytket.extensions.qiskit import AerBackend

warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

First, we configure the backend and initialize a `TketModel` object and load the model from the file saved last time.

In [4]:
backend = AerBackend()
backend_config = {
    'backend': backend,
    'compilation': backend.default_compilation_pass(2),
    'shots': 1024
}

model = TketModel.from_checkpoint("model.lt", backend_config)

TypeError: Model.from_checkpoint() takes 2 positional arguments but 3 were given